# 🚀 Django DepEd Memo System - Code Improvements Guide

This interactive notebook provides **step-by-step code improvements** for your Django project.

## 📑 Table of Contents
1. [Critical Bug Fixes](#critical-bugs)
2. [Security Improvements](#security)
3. [Code Quality Enhancements](#quality)
4. [Performance Optimizations](#performance)
5. [Best Practices](#best-practices)

---
## 🔴 CRITICAL BUG #1: Update View Creates Tuples

### Location: `a1_admin_user/views.py` lines 49-51

### ❌ Current Broken Code

In [ ]:
# BROKEN - DO NOT USE
def memo_update_views(request, id):
    if request.method == 'POST':
        title = request.POST.get('title')
        description = request.POST.get('description')
        reference_data = request.POST.get('reference_data')
        
        memo = MemoTable.objects.get(id=id)
        
        # ❌ BUG: Trailing commas create tuples!
        memo.title = title,              # Creates tuple: ('New Title',)
        memo.description = description,  # Creates tuple: ('Description',)
        reference_data = reference_data, # Doesn't even save to memo!
        
        memo.save()
        return redirect('memo_views')

### ✅ Fixed Code

In [ ]:
# FIXED VERSION
from django.shortcuts import render, get_object_or_404, redirect
from django.contrib import messages
from b2_dep_model.models import MemoTable

def memo_update_views(request, id):
    memo = get_object_or_404(MemoTable, id=id)
    
    if request.method == 'POST':
        title = request.POST.get('title')
        description = request.POST.get('description')
        reference_data = request.POST.get('reference_data')
        file = request.FILES.get('file')
        
        # ✅ FIX: Remove trailing commas
        memo.title = title
        memo.description = description
        memo.reference_data = reference_data
        
        if file:
            memo.file = file
        
        memo.save()
        messages.success(request, 'Memo updated successfully!')
        return redirect('memo_views')
    
    return render(request, 'for_admin/page3.html', {'memo': memo})

### 🧪 Test the difference

In [ ]:
# Demonstration of the bug
class TestMemo:
    def __init__(self):
        self.title = "Original"
        self.description = "Original Description"

memo = TestMemo()

# WRONG WAY (with comma)
memo.title = "New Title",
print(f"With comma: {memo.title} (type: {type(memo.title)})")

# RIGHT WAY (without comma)
memo.description = "New Description"
print(f"Without comma: {memo.description} (type: {type(memo.description)})")

---
## 🔴 CRITICAL BUG #2: Month/Year Format Mismatch

### Location: `a1_admin_user/views.py` lines 62-63

### ❌ Current Broken Code

In [ ]:
# BROKEN FORMAT
from datetime import datetime

# ❌ This produces '11-29' not 'November'
month = datetime.now().strftime("%m-%d")
print(f"Month: {month}")  # Output: "11-29"

# ❌ This produces '25' not '2025'
year = datetime.now().strftime("%y")
print(f"Year: {year}")    # Output: "25"

# But your search expects:
# Month: "November"
# Year: "2025"
# They won't match!

### ✅ Fixed Code

In [ ]:
# FIXED VERSION
from datetime import datetime

# ✅ Correct format for month name
month = datetime.now().strftime("%B")
print(f"Month: {month}")  # Output: "November"

# ✅ Correct format for full year
year = datetime.now().strftime("%Y")
print(f"Year: {year}")    # Output: "2025"

# Updated upload view
def memo_upload_views(request):
    if request.method == 'POST':
        title = request.POST.get('title')
        description = request.POST.get('description')
        reference_data = request.POST.get('reference')
        file = request.FILES.get('file')
        
        # ✅ Correct date formatting
        month = datetime.now().strftime("%B")   # "November"
        year = datetime.now().strftime("%Y")    # "2025"
        
        MemoTable.objects.update(recent=False)
        
        MemoTable.objects.create(
            title=title,
            description=description,
            month=month,
            year=year,
            reference_data=reference_data,
            file=file
        )
        return redirect('memo_views')
    
    return render(request, 'for_admin/page3.html')

---
## 🔐 SECURITY #1: Add Authentication to Admin Views

### Current Problem
Anyone can access admin functions without logging in!

### ✅ Solution: Add Login Required Decorators

In [ ]:
# Add to a1_admin_user/views.py
from django.contrib.auth.decorators import login_required, user_passes_test
from django.contrib import messages
from django.shortcuts import redirect

# Check if user is staff
def is_staff_user(user):
    return user.is_authenticated and user.is_staff

# Apply to all admin views
@login_required
@user_passes_test(is_staff_user, login_url='/login/')
def memo_views(request):
    memo_list = MemoTable.objects.filter(recent=False)
    # ... rest of code

@login_required
@user_passes_test(is_staff_user, login_url='/login/')
def memo_upload_views(request):
    # ... implementation
    pass

@login_required
@user_passes_test(is_staff_user, login_url='/login/')
def memo_update_views(request, id):
    # ... implementation
    pass

@login_required
@user_passes_test(is_staff_user, login_url='/login/')
def memo_delete_view(request):
    # ... implementation
    pass

### Add to settings.py

In [ ]:
# deped/settings.py
LOGIN_URL = '/admin/login/'
LOGIN_REDIRECT_URL = '/admin-page/'
LOGOUT_REDIRECT_URL = '/'

---
## 🔐 SECURITY #2: File Upload Validation

### Create a validators.py file

In [ ]:
# Create: a1_admin_user/validators.py
from django.core.exceptions import ValidationError
import os
import magic  # pip install python-magic

def validate_pdf_file(file):
    """
    Validate uploaded PDF file for:
    - File extension
    - File size
    - MIME type
    """
    # Maximum file size: 10MB
    MAX_FILE_SIZE = 10 * 1024 * 1024
    
    # Check file extension
    ext = os.path.splitext(file.name)[1].lower()
    if ext != '.pdf':
        raise ValidationError(
            f'Invalid file extension: {ext}. Only PDF files are allowed.'
        )
    
    # Check file size
    if file.size > MAX_FILE_SIZE:
        raise ValidationError(
            f'File size ({file.size / 1024 / 1024:.2f}MB) exceeds maximum (10MB)'
        )
    
    # Check MIME type using content
    if hasattr(file, 'read'):
        file_content = file.read(2048)
        file.seek(0)  # Reset file pointer
        
        mime = magic.from_buffer(file_content, mime=True)
        if mime != 'application/pdf':
            raise ValidationError(
                f'Invalid file type: {mime}. Expected application/pdf'
            )
    
    return file


def validate_image_file(file):
    """
    Validate uploaded image file for admin profiles
    """
    MAX_FILE_SIZE = 5 * 1024 * 1024  # 5MB
    ALLOWED_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.gif']
    ALLOWED_MIMES = ['image/jpeg', 'image/png', 'image/gif']
    
    ext = os.path.splitext(file.name)[1].lower()
    if ext not in ALLOWED_EXTENSIONS:
        raise ValidationError(
            f'Invalid file extension: {ext}. Allowed: {ALLOWED_EXTENSIONS}'
        )
    
    if file.size > MAX_FILE_SIZE:
        raise ValidationError(
            f'File size exceeds maximum (5MB)'
        )
    
    return file

### Use validation in views

In [ ]:
# In a1_admin_user/views.py
from .validators import validate_pdf_file
from django.core.exceptions import ValidationError

def memo_upload_views(request):
    if request.method == 'POST':
        file = request.FILES.get('file')
        
        if not file:
            messages.error(request, 'Please select a file to upload')
            return redirect('memo_upload_views')
        
        # ✅ Validate file before saving
        try:
            validate_pdf_file(file)
        except ValidationError as e:
            messages.error(request, str(e))
            return redirect('memo_upload_views')
        
        # Continue with creation...
        MemoTable.objects.create(
            title=title,
            description=description,
            reference_data=reference_data,
            file=file,
            month=month,
            year=year
        )
        
        messages.success(request, 'Memo uploaded successfully!')
        return redirect('memo_views')

---
## 📝 CODE QUALITY #1: Use Django Forms

### Create forms.py

In [ ]:
# Create: a1_admin_user/forms.py
from django import forms
from b2_dep_model.models import MemoTable
from .validators import validate_pdf_file

class MemoUploadForm(forms.ModelForm):
    """
    Form for uploading new memos
    """
    class Meta:
        model = MemoTable
        fields = ['title', 'description', 'reference_data', 'file']
        widgets = {
            'title': forms.TextInput(attrs={
                'class': 'form-control',
                'placeholder': 'Enter memo title',
                'maxlength': 255
            }),
            'description': forms.Textarea(attrs={
                'class': 'form-control',
                'placeholder': 'Enter description',
                'rows': 4
            }),
            'reference_data': forms.TextInput(attrs={
                'class': 'form-control',
                'placeholder': 'e.g., REF-2025-001'
            }),
            'file': forms.FileInput(attrs={
                'class': 'form-control',
                'accept': '.pdf'
            })
        }
    
    def clean_reference_data(self):
        """Ensure reference is unique"""
        reference = self.cleaned_data['reference_data']
        
        if MemoTable.objects.filter(reference_data=reference).exists():
            raise forms.ValidationError(
                f'A memo with reference "{reference}" already exists.'
            )
        
        return reference
    
    def clean_file(self):
        """Validate uploaded file"""
        file = self.cleaned_data.get('file')
        if file:
            validate_pdf_file(file)
        return file


class MemoUpdateForm(forms.ModelForm):
    """
    Form for updating existing memos
    """
    class Meta:
        model = MemoTable
        fields = ['title', 'description', 'reference_data', 'file']
        widgets = {
            'title': forms.TextInput(attrs={'class': 'form-control'}),
            'description': forms.Textarea(attrs={'class': 'form-control', 'rows': 4}),
            'reference_data': forms.TextInput(attrs={'class': 'form-control'}),
            'file': forms.FileInput(attrs={'class': 'form-control', 'accept': '.pdf'})
        }
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Make file field optional when updating
        self.fields['file'].required = False
    
    def clean_file(self):
        file = self.cleaned_data.get('file')
        if file:
            validate_pdf_file(file)
        return file

### Use forms in views

In [ ]:
# Updated a1_admin_user/views.py
from .forms import MemoUploadForm, MemoUpdateForm
from datetime import datetime

def memo_upload_views(request):
    if request.method == 'POST':
        form = MemoUploadForm(request.POST, request.FILES)
        
        if form.is_valid():
            # Set all existing memos to not recent
            MemoTable.objects.update(recent=False)
            
            # Create new memo
            memo = form.save(commit=False)
            memo.month = datetime.now().strftime("%B")
            memo.year = datetime.now().strftime("%Y")
            memo.recent = True
            memo.save()
            
            messages.success(request, 'Memo uploaded successfully!')
            return redirect('memo_views')
        else:
            # Form has validation errors
            messages.error(request, 'Please correct the errors below.')
    else:
        form = MemoUploadForm()
    
    return render(request, 'for_admin/page3.html', {'form': form})


def memo_update_views(request, id):
    memo = get_object_or_404(MemoTable, id=id)
    
    if request.method == 'POST':
        form = MemoUpdateForm(request.POST, request.FILES, instance=memo)
        
        if form.is_valid():
            form.save()
            messages.success(request, 'Memo updated successfully!')
            return redirect('memo_views')
        else:
            messages.error(request, 'Please correct the errors below.')
    else:
        form = MemoUpdateForm(instance=memo)
    
    return render(request, 'for_admin/page3.html', {'form': form, 'memo': memo})

---
## 🗄️ DATABASE #1: Add Model Constraints

### Updated models.py

In [ ]:
# b2_dep_model/models.py - UPDATED VERSION
from django.db import models
from django.core.validators import FileExtensionValidator
from django.utils import timezone

class MemoTable(models.Model):
    title = models.CharField(
        max_length=255,
        help_text="Memo title (max 255 characters)"
    )
    description = models.TextField(
        max_length=1000,  # Increased from 255
        help_text="Detailed description of the memo"
    )
    reference_data = models.CharField(
        max_length=50,
        unique=True,  # ✅ Prevent duplicate references
        db_index=True,  # ✅ Faster lookups
        help_text="Unique reference number (e.g., REF-2025-001)"
    )
    month = models.CharField(
        max_length=50,
        db_index=True  # ✅ Index for search performance
    )
    year = models.CharField(
        max_length=50,
        db_index=True  # ✅ Index for search performance
    )
    recent = models.BooleanField(
        default=True,
        db_index=True  # ✅ Index for filtering
    )
    file = models.FileField(
        upload_to='pdf/',
        validators=[FileExtensionValidator(allowed_extensions=['pdf'])],
        help_text="PDF file only, max 10MB"
    )
    
    # ✅ Add timestamp fields
    created_at = models.DateTimeField(
        auto_now_add=True,
        help_text="When the memo was created"
    )
    updated_at = models.DateTimeField(
        auto_now=True,
        help_text="When the memo was last updated"
    )
    
    class Meta:
        db_table = 'memo_table'
        ordering = ['-created_at']  # ✅ Order by newest first
        verbose_name = 'Memo'
        verbose_name_plural = 'Memos'
        
        # ✅ Add composite indexes for common queries
        indexes = [
            models.Index(fields=['month', 'year']),
            models.Index(fields=['recent', '-created_at']),
            models.Index(fields=['title']),
        ]
    
    def __str__(self):
        return f"{self.reference_data} - {self.title}"
    
    def __repr__(self):
        return f"<Memo: {self.reference_data}>"

### Run migrations after model changes

In [ ]:
# Run these commands in your terminal:
# python manage.py makemigrations
# python manage.py migrate

# Or run from notebook:
import subprocess
import os

os.chdir(r"c:\Users\francis\OneDrive\Desktop\DJANGO PROJECT\Deped")

# Create migrations
result = subprocess.run(
    ['python', 'manage.py', 'makemigrations'],
    capture_output=True,
    text=True
)
print(result.stdout)

# Apply migrations
result = subprocess.run(
    ['python', 'manage.py', 'migrate'],
    capture_output=True,
    text=True
)
print(result.stdout)

---
## ⚡ PERFORMANCE #1: Optimize Queries

### Current inefficient code

In [ ]:
# ❌ SLOW - retrieves all fields
memo_list = MemoTable.objects.filter(recent=False).values(
    'id', 'title', 'description', 'reference_data', 'month', 'year', 'file'
)

# Issues:
# 1. Using .values() loses ORM benefits
# 2. No query optimization
# 3. May retrieve unused fields

### ✅ Optimized code

In [ ]:
# ✅ FAST - only retrieve needed fields
memo_list = MemoTable.objects.filter(
    recent=False
).only(
    'id', 'title', 'description', 'reference_data', 'month', 'year', 'file'
).order_by('-created_at')

# Benefits:
# 1. Still get model instances
# 2. Only selected fields in query
# 3. Can access methods and properties

### Add caching for frequently accessed data

In [ ]:
# Add to settings.py first:
CACHES = {
    'default': {
        'BACKEND': 'django.core.cache.backends.locmem.LocMemCache',
        'LOCATION': 'deped-memo-cache',
        'OPTIONS': {
            'MAX_ENTRIES': 1000
        }
    }
}

# Then in views:
from django.core.cache import cache
from django.views.decorators.cache import cache_page

# Cache the entire view for 15 minutes
@cache_page(60 * 15)
def memo_page1_view(request):
    # ... your code
    pass

# Or cache specific queries
def get_recent_memo():
    cache_key = 'recent_memo'
    memo = cache.get(cache_key)
    
    if memo is None:
        memo = MemoTable.objects.filter(recent=True).first()
        cache.set(cache_key, memo, 60 * 15)  # 15 minutes
    
    return memo

---
## 🔒 SECURITY #3: Environment Variables

### Install python-decouple

In [ ]:
# Run in terminal:
# pip install python-decouple

!pip install python-decouple

### Create .env file

In [ ]:
# Create file: .env (in project root)
# Add this content:

"""
DEBUG=True
SECRET_KEY=your-new-secret-key-here-change-this
ALLOWED_HOSTS=localhost,127.0.0.1

# Database (for production)
DB_NAME=deped_db
DB_USER=postgres
DB_PASSWORD=your-password
DB_HOST=localhost
DB_PORT=5432
"""

# IMPORTANT: Add .env to .gitignore!
print("Add to .gitignore:")
print(".env")
print("*.pyc")
print("__pycache__/")
print("db.sqlite3")
print("media/")

### Update settings.py

In [ ]:
# deped/settings.py - UPDATED
from pathlib import Path
from decouple import config, Csv
import os

BASE_DIR = Path(__file__).resolve().parent.parent

# ✅ Use environment variables
SECRET_KEY = config('SECRET_KEY')
DEBUG = config('DEBUG', default=False, cast=bool)
ALLOWED_HOSTS = config('ALLOWED_HOSTS', default='', cast=Csv())

# Database configuration
if DEBUG:
    # Development: SQLite
    DATABASES = {
        'default': {
            'ENGINE': 'django.db.backends.sqlite3',
            'NAME': BASE_DIR / 'db.sqlite3',
        }
    }
else:
    # Production: PostgreSQL
    DATABASES = {
        'default': {
            'ENGINE': 'django.db.backends.postgresql',
            'NAME': config('DB_NAME'),
            'USER': config('DB_USER'),
            'PASSWORD': config('DB_PASSWORD'),
            'HOST': config('DB_HOST'),
            'PORT': config('DB_PORT', default='5432'),
        }
    }

# Security settings for production
if not DEBUG:
    SECURE_SSL_REDIRECT = True
    SESSION_COOKIE_SECURE = True
    CSRF_COOKIE_SECURE = True
    SECURE_BROWSER_XSS_FILTER = True
    SECURE_CONTENT_TYPE_NOSNIFF = True

---
## 📊 SUMMARY: Implementation Checklist

### Quick Fixes (Do Now)
- [ ] Fix trailing commas in `memo_update_views` (Bug #1)
- [ ] Fix month/year format in `memo_upload_views` (Bug #2)
- [ ] Remove all `print()` statements
- [ ] Remove commented-out code

### Week 1: Critical Security
- [ ] Add `@login_required` to all admin views
- [ ] Create `validators.py` for file validation
- [ ] Install and configure `python-decouple`
- [ ] Move SECRET_KEY to `.env`
- [ ] Add `.env` to `.gitignore`

### Week 2: Code Quality
- [ ] Create `forms.py` with Django forms
- [ ] Update views to use forms
- [ ] Add model constraints (unique, indexes)
- [ ] Run and test migrations
- [ ] Set up logging instead of print

### Week 3: Performance
- [ ] Add database indexes
- [ ] Optimize queries with `.only()`
- [ ] Implement caching
- [ ] Test performance improvements

### Week 4: Testing & Polish
- [ ] Write unit tests
- [ ] Add integration tests
- [ ] Create requirements.txt
- [ ] Update documentation
- [ ] Code review and refactoring

---
## 🎓 Additional Resources

### Django Documentation
- [Forms](https://docs.djangoproject.com/en/5.1/topics/forms/)
- [Authentication](https://docs.djangoproject.com/en/5.1/topics/auth/)
- [File Uploads](https://docs.djangoproject.com/en/5.1/topics/http/file-uploads/)
- [Database Optimization](https://docs.djangoproject.com/en/5.1/topics/db/optimization/)
- [Security](https://docs.djangoproject.com/en/5.1/topics/security/)

### Best Practices
- [Two Scoops of Django](https://www.feldroy.com/books/two-scoops-of-django-3-x)
- [Django Best Practices](https://django-best-practices.readthedocs.io/)

---

**Created**: November 29, 2025  
**Author**: AI Code Review System  
**Project**: DepEd Memo Management System